# SC02_Gruppe_10 | Basic Cost of Living
## Problem Description
The cost of living increases faster than the wages increase leading to a higher demand of cost of living optimization.
The two biggest positions in the cost of living are rent and health care.

### Thesis
By visualizing the basic cost of living per canton (in a heatmap) the choice of canton of habitat for people/families willing to move is simlified.

In [1]:
# init
from bs4 import BeautifulSoup       # scrape a website
import json
# import matplotlib.pyplot as plt
import numpy as np                  # the vector victor
import os                           # operating system specific tasks incl. file handling
import pandas as pd                 # pandas - handle dataframes (2d)
from pprint import pprint as pp     # make output of lists, dicts and co more readable
import urllib.request               # pythons built in way to get stuff from the web
import requests                     # !! probably not needed. urllib.request might be good enough for our case
# import seaborn as sns
import zipfile                      # for looking into and extracting files from a zip-archive

In [2]:
# misc
home_base = os.getcwd()

In [3]:
# logging
import traceback
import logging
logger = logging.getLogger(__name__)
logger.setLevel("DEBUG")

# log handlers (how/whereto output logs)
console_handler = logging.StreamHandler() # log to console
file_handler = logging.FileHandler("logs/app.log", mode="a", encoding="utf-8") # log more important info to a file
logger.addHandler(console_handler)
logger.addHandler(file_handler)

# format currently the same for console and file but might make sense to change in the future
formatter = logging.Formatter(
   "{asctime}:{levelname} - {name}:{funcName}: {message}",
    style="{",
    datefmt="%Y-%m-%d_%H%M%S",
)
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# only log info and higher to the file
file_handler.setLevel("INFO")

logger.debug(logger.handlers)

logger.info(f"Program initialized at {home_base=}. Dependencies loaded.")

2024-10-13_141439:DEBUG - __main__:<module>: [<StreamHandler stderr (NOTSET)>, <FileHandler /home/carl/DAS_DataScience/CAS_InformationEngineering/SC02_Gruppe_10/SC02_Gruppe_10/logs/app.log (INFO)>]
2024-10-13_141439:INFO - __main__:<module>: Program initialized at home_base='/home/carl/DAS_DataScience/CAS_InformationEngineering/SC02_Gruppe_10/SC02_Gruppe_10'. Dependencies loaded.


In [4]:
class Data:
  """ Data class for the project """
  def get_data_frame(self):
    """ just some plaseholder methods to define the structure """
    pass

  def get_columns(self):
    """ just some plaseholder methods to define the structure """
    pass

  def get_rows(self):
    """ just some plaseholder methods to define the structure """
    pass



In [5]:
class File:
    """ common methods for handling files """
    def get_content_from_url(url: str) -> str:
        """ get the raw contents from some http url and return it without parsing a thing """
        return requests.get(url).content
    
    def download(url: str, file_path=None):
        """ download a file from the internet and store it locally """
        try:
            path, headers = urllib.request.urlretrieve(url,file_path)
            logger.debug("File downloaded to path=%s", path)
        except Exception as e:
            logger.exception("FileDownloadError")
            # raise
        # for name, value in headers.items():
        #     print(name, value)


# some testing of the class and its methods
url = (
    "https://api.worldbank.org/v2/en/indicator/"
    "NY.GDP.MKTP.CD?downloadformat=csv"
)
filename = "gdp_by_country.zip"
File.download(url)

2024-10-13_141514:DEBUG - __main__:download: File downloaded to path=/tmp/tmp6ow5vrql


In [95]:
# just a little test of the 'help'-information
#my_data = Data()
#help(Data)

In [6]:
# ##################################
# oritinal rent data
# ##################################
source_price_rent_url = "https://dam-api.bfs.admin.ch/hub/api/dam/assets/30885375/master"
xls = source_price_rent_data = pd.ExcelFile(source_price_rent_url)

# check which years we have available (tab-name = year of statistic)
print(xls.sheet_names)

# To Do: Loop through the files and create a multi-indexed dataframe off of it
# to read just one sheet to dataframe:
#df = pd.read_excel(file_name, sheet_name="house")


['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2003', '2000']


In [7]:
# ##################################
# oritinal health care data
# ##################################
source_price_health_care_url = "https://ckan.opendata.swiss/api/3/action/package_show?id=health-insurance-premiums"
price_health_care_map_name = 'Prämien_CH.xlsx'
with urllib.request.urlopen(source_price_health_care_url) as url:
    data = json.load(url)
    resources = data['result']['resources']
    for resource in resources:
        # we are only interested in Prämien_CH.xlsx, which we can either find directly in the resoureces
        # and sometime within a zip archive
        if 'ZIP' == resource['format']:
            print(f"{resource['format']} {resource['name']['en']} | {resource['download_url']} | extraction needed ... .. .")
            
            # TO DO: handle the zip, which we have actually (temporarily) downloaded, not a static zip as in this exampel
            # zip file handler
            # zip = zipfile.ZipFile('filename.zip')

            # # list available files in the container
            # print (zip.namelist())

            # # extract a specific file from the zip container
            # f = zip.open("file_inside_zip.txt")

            # # save the extraced file 
            # content = f.read()
            # f = open('file_inside_zip.extracted.txt', 'wb')
            # f.write(content)
            # f.close()

        elif price_health_care_map_name == resource['name']['en']:
            print(f"{resource['format']} {resource['name']['en']} | {resource['download_url']}")
              

XLS Prämien_CH.xlsx | https://bag-files.opendata.swiss/owncloud/index.php/s/D3bD8NJHkXkzaZ7
ZIP Archiv_Praemien_2024.zip | https://bag-files.opendata.swiss/owncloud/index.php/s/5qptAxHT7Pii9WL | extraction needed ... .. .


FileNotFoundError: [Errno 2] No such file or directory: 'filename.zip'